In [1]:
from transformers import TFAutoModelForMaskedLM

model_checkpoint = "allenai/scibert_scivocab_uncased"
model = TFAutoModelForMaskedLM.from_pretrained(model_checkpoint, from_pt=True)

model.summary()

2023-05-25 22:22:47.577293: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-25 22:22:49.147636: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /shared/centos7/cuda/11.7/lib64:/shared/centos7/nodejs/14.15.4/lib
2023-05-25 22:22:49.148458: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /shared/centos7/cuda/11.7/lib64:/shared/centos7/nodejs/

ResourceExhaustedError: Exception encountered when calling layer 'bert' (type TFBertMainLayer).

{{function_node __wrapped__Mul_device_/job:localhost/replica:0/task:0/device:GPU:0}} failed to allocate memory [Op:Mul]

Call arguments received by layer 'bert' (type TFBertMainLayer):
  • input_ids=tf.Tensor(shape=(3, 5), dtype=int32)
  • attention_mask=None
  • token_type_ids=None
  • position_ids=None
  • head_mask=None
  • inputs_embeds=None
  • encoder_hidden_states=None
  • encoder_attention_mask=None
  • past_key_values=None
  • use_cache=None
  • output_attentions=False
  • output_hidden_states=False
  • return_dict=True
  • training=False

In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [120]:
from datasets import load_dataset
dataset = load_dataset("csv", data_files={"train":"acronym_train.csv", "test":"acronym_test.csv"})

dataset

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /home/yue.r/.cache/huggingface/datasets/csv/default-6309002f5eafbfbd/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['masked', 'outputs'],
        num_rows: 504
    })
    test: Dataset({
        features: ['masked', 'outputs'],
        num_rows: 168
    })
})

In [7]:
dataset["train"][20]

{'masked': 'femoral artery ([MASK])', 'outputs': 'femoral artery (FA)'}

In [8]:
sample = dataset["train"].shuffle(seed=42).select(range(3))

for row in sample:
    print(f"'>>> Input IDs: {row['masked']}")
    print(f"'>>> Outputs: {row['outputs']}")

Loading cached shuffled indices for dataset at /home/yue.r/.cache/huggingface/datasets/csv/default-863b486ab570f9ce/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-011f33912d36de24.arrow


'>>> Input IDs: transcription factor ([MASK])
'>>> Outputs: transcription factor (TF)
'>>> Input IDs: minimal inhibitory concentration ([MASK])
'>>> Outputs: minimal inhibitory concentration (MIC)
'>>> Input IDs: high level cephalosporinase ([MASK])
'>>> Outputs: high level cephalosporinase (HL-CASE)


In [9]:
def tokenize_function(examples):
    result = tokenizer(examples["masked"], padding='max_length', max_length=30)
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    
    return result

def tokenize_labels(examples):
    result = tokenizer(examples["outputs"], padding='max_length', max_length=30)
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
        result["labels"] = result["input_ids"].copy()
    
    return result

In [10]:
tokenized_dataset = dataset.map(
    tokenize_labels, batched=True, remove_columns=["outputs"]
)

tokenized_dataset = tokenized_dataset.map(
    tokenize_function, batched=True, remove_columns=["word_ids", "masked", "input_ids", "token_type_ids", "attention_mask"]
)

tokenized_dataset

Loading cached processed dataset at /home/yue.r/.cache/huggingface/datasets/csv/default-863b486ab570f9ce/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-468df5f81e0d0012.arrow
Loading cached processed dataset at /home/yue.r/.cache/huggingface/datasets/csv/default-863b486ab570f9ce/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-b2be5607922cfa08.arrow
Loading cached processed dataset at /home/yue.r/.cache/huggingface/datasets/csv/default-863b486ab570f9ce/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-f5f778dab4111420.arrow
Loading cached processed dataset at /home/yue.r/.cache/huggingface/datasets/csv/default-863b486ab570f9ce/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-0010c80a190cd5b3.arrow


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 504
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 168
    })
})

In [14]:
from transformers import DataCollatorWithPadding

tf_train = tokenized_dataset["train"].to_tf_dataset(
    columns=['input_ids', 'token_type_ids', 'attention_mask'],
    label_cols=["labels"],
    shuffle = True,
    batch_size=32,
)

tf_test = tokenized_dataset["test"].to_tf_dataset(
    columns=['input_ids', 'token_type_ids', 'attention_mask'],
    label_cols=["labels"],
    shuffle = True,
    batch_size=32,
)

tf_train

/home/yue.r/miniconda/envs/tf_v2/lib/python3.7/site-packages/datasets/arrow_dataset.py:396: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  FutureWarning,


<PrefetchDataset element_spec=({'input_ids': TensorSpec(shape=(None, 30), dtype=tf.int64, name=None), 'token_type_ids': TensorSpec(shape=(None, 30), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(None, 30), dtype=tf.int64, name=None)}, TensorSpec(shape=(None, 30), dtype=tf.int64, name=None))>

In [15]:
from transformers import create_optimizer
import tensorflow as tf

num_train_steps = len(tf_train)
optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=1_000,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01
)

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

model.compile(optimizer=optimizer, loss=loss, metrics=["accuracy"])

tf.keras.mixed_precision.set_global_policy("mixed_float16")

model_name = model_checkpoint.split("/")[-1]

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: Tesla V100-SXM2-32GB, compute capability 7.0


In [16]:
model.fit(tf_train, epochs=25)

Epoch 1/25
16/16 [==============================] - 21s 123ms/step - loss: 13.0101 - accuracy: 0.1733
Epoch 2/25
16/16 [==============================] - 2s 119ms/step - loss: 9.4692 - accuracy: 0.1748
Epoch 3/25
16/16 [==============================] - 2s 121ms/step - loss: 2.8866 - accuracy: 0.6587
Epoch 4/25
16/16 [==============================] - 2s 121ms/step - loss: 1.4584 - accuracy: 0.8485
Epoch 5/25
16/16 [==============================] - 2s 120ms/step - loss: 0.8486 - accuracy: 0.8773
Epoch 6/25
16/16 [==============================] - 2s 121ms/step - loss: 0.4981 - accuracy: 0.9063
Epoch 7/25
16/16 [==============================] - 2s 121ms/step - loss: 0.3330 - accuracy: 0.9290
Epoch 8/25
16/16 [==============================] - 2s 122ms/step - loss: 0.2713 - accuracy: 0.9318
Epoch 9/25
16/16 [==============================] - 2s 123ms/step - loss: 0.2340 - accuracy: 0.9354
Epoch 10/25
16/16 [==============================] - 2s 123ms/step - loss: 0.2110 - accuracy: 0.93

In [121]:
def get_length(lf):
    count = 0
    for word in lf.split():
        if ("(" not in word and
            ")" not in word and
            word != "and" and
            word != "or" and
            word != "of"):
            if "-" in word and word[-1] != "- ":
                count += 1
            count +=1
            
    return count

In [122]:
def predict(lf):
    prediction = None
    inputs = tokenizer(lf, return_tensors="np")
    token_logits = model(**inputs).logits
    
    mask_token_index = np.argwhere(inputs["input_ids"] == tokenizer.mask_token_id)[0, 1]
    mask_token_logits = token_logits[0, mask_token_index, :]

    top_token_index_1 = np.argsort(-mask_token_logits)[0]
    
    lf = lf.replace(tokenizer.mask_token, tokenizer.decode(top_token_index_1).upper())
    
    phrase_length = get_length(lf)
    
    if len(tokenizer.decode(top_token_index_1)) < phrase_length:
        lf = lf.replace(")", " " + tokenizer.mask_token + ")")
        inputs = tokenizer(lf, return_tensors="np")
        token_logits = model(**inputs).logits
        mask_token_index2 = mask_token_index + 1
        mask_token_logits2 = token_logits[0, mask_token_index2, :]
        top_token_index_2 = np.argsort(-mask_token_logits2)[0]
        prediction = lf.replace(" " + tokenizer.mask_token, tokenizer.decode(top_token_index_2)[2:].upper())
        
    if prediction == None:
        prediction = lf
        
    return prediction

In [123]:
results = []
for i in range(len(dataset["test"])):
    results.append(predict(dataset["test"][i]["masked"]))

In [124]:
import csv

fields = ["prediction", "gold_labels"]

gold_labels = dataset["test"]["outputs"]

data = zip(results, gold_labels)

with open('predictions.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(fields)
    for item in data:
        writer.writerow(item)

In [21]:
test_results = model.predict(tf_test)
tokenized_dataset["test"]

6/6 [==============================] - 3s 44ms/step


2023-05-24 23:09:17.727220: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 626774400 exceeds 10% of free system memory.


Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
    num_rows: 168
})